In [125]:
import pandas as pd
import os
import seaborn as sns
import scipy.stats as sps
from IPython.core.display import display, HTML
import swifter
import multiprocessing as mp
import numpy as np
pd.set_option('precision', 2)

In [126]:
wilcoxgreater = lambda data: sps.wilcoxon(data, alternative='greater').pvalue

def readcsv(loc):
    return pd.read_csv("{0}/{1}".format(loc[0], loc[1]))

def readScatteredResults(dset, dire="bayesian_results"):
    n = 8
    pool = mp.Pool(n)
    files = os.listdir(dire)
    files = list(filter(lambda name: ".csv" in name and dset in name, files))
    args = list(zip([dire for i in range(len(files))], files))
    cs = max(int(np.ceil(len(files)/(4*n))),1)
    reading = pool.map(readcsv, args, chunksize=cs)
    pool.close()
    pool.join()
    return pd.concat(reading)

## Aggregating result .csv files

In [127]:
locations = ["results"] # folder containing .csv files
dsets = ["infantgts"] # Dset name on .csv file name to look for, #['tourism', 'infantgts']

In [128]:
total = pd.DataFrame()
for location in locations:
    for key in dsets:
        
        print(key)
        results = readScatteredResults(key, dire=location).sort_values(['h', 'iTest', 'fmethod'])    
        results = results.loc[:, ~results.columns.duplicated()]
        khOneMask = results['khOne'] == True
        results.loc[khOneMask, 'kh'] = 1
        results.loc[~khOneMask, 'kh'] = results.loc[khOneMask, 'h'].values
        total = total.append(results)
    
total.loc[total['khOne']==True,'run']='kh=1'
total.loc[total['khOne']==False,'run']='kh=h'
total = total.set_index(['dset', 'h', 'fmethod', "kh", "run", "iTest"])
print(len(total))

infantgts
388


In [129]:
grouper_kh = 'run' #kh
total = total.groupby([grouper_kh, 'dset', 'h', 'fmethod', 'iTest', "seed"]).agg('mean').reset_index()
total = total.set_index([grouper_kh, 'dset', 'h', 'fmethod'])
print(len(total))
total

388


iTest  seed  khOne  rmseMeanAll_Base  \
run  dset      h fmethod                                         
kh=1 infantgts 1 ets          1     0   True             21.58   
                 ets          2     0   True             20.25   
                 ets          3     0   True             12.46   
                 ets          4     0   True             34.29   
                 ets          5     0   True             46.71   
...                         ...   ...    ...               ...   
kh=h infantgts 4 ets         43     0  False             25.41   
                 ets         44     0  False             34.29   
                 ets         45     0  False            108.98   
                 ets         46     0  False             70.32   
                 ets         47     0  False             43.53   

                          rmseMedianAll_Base  maeMeanAll_Base  \
run  dset      h fmethod                                        
kh=1 infantgts 1 ets                   21.58            15.27   
                 ets                   20.25            14.37   
                 ets                   12.46            10.50   
                 ets                   34.29            20.55   
                 ets                   46.71            27.37   
...                                      ...              ...   
kh=h infantgts 4 ets                   25.41            20.34   
                 ets                   34.29            23.20   
                 ets                  108.98            64.32   
                 ets                   70.32            44.22   
                 ets                   43.53            27.90   

                          maeMedianAll_Base  esAll_Base  rmseMeanUpper_Base  \
run  dset      h fmethod                                                      
kh=1 infantgts 1 ets                  15.27      113.28               30.42   
                 ets                  14.37      111.85               27.20   
                 ets                  10.50      102.87               15.07   
                 ets                  20.55      138.09               50.46   
                 ets                  27.37      171.23               68.27   
...                                     ...         ...                 ...   
kh=h infantgts 4 ets                  20.34      224.82               31.33   
                 ets                  23.20      236.96               42.88   
                 ets                  64.32      399.24              157.09   
                 ets                  44.22      307.03               89.41   
                 ets                  27.90      257.30               56.00   

                          rmseMedianUpper_Base  ...  \
run  dset      h fmethod                        ...   
kh=1 infantgts 1 ets                     30.42  ...   
                 ets                     27.20  ...   
                 ets                     15.07  ...   
                 ets                     50.46  ...   
                 ets                     68.27  ...   
...                                        ...  ...   
kh=h infantgts 4 ets                     31.33  ...   
                 ets                     42.88  ...   
                 ets                    157.09  ...   
                 ets                     89.41  ...   
                 ets                     56.00  ...   

                          rmseMeanDepth3.State._Indep  \
run  dset      h fmethod                                
kh=1 infantgts 1 ets                            22.70   
                 ets                            21.87   
                 ets                            12.18   
                 ets                            24.59   
                 ets                            43.41   
...                                               ...   
kh=h infantgts 4 ets                            29.90   
                 ets                            43.00   
       

## Counting experiment members

* asserting each dataset has the correct number of experiments
* asserting all columns are correctly filled

In [130]:
counter = total.groupby(['dset',grouper_kh, 'h', 'fmethod']).agg({'rmseMeanAll_Base':'count', 'seed': 'nunique'})
counter.filter(regex='infantgts',axis=0)

rmseMeanAll_Base  seed
dset      run  h fmethod                        
infantgts kh=1 1 ets                    50     1
               2 ets                    49     1
               3 ets                    48     1
               4 ets                    47     1
          kh=h 1 ets                    50     1
               2 ets                    49     1
               3 ets                    48     1
               4 ets                    47     1

## Filtering metric to be studied and mapping columns to method

In [131]:
variable='esAll'
filtered = total.filter(regex=variable)
all_ = filtered
map_ = {"{0}_Base".format(variable): "Base",
        "{0}_BottomUp".format(variable): "BU",
       "{0}_Corr".format(variable): "pMinT",
       "{0}_Indep".format(variable): "LG"}

## Energy Scores

In [132]:
dset = 'infantgts'
m = 'arima|ets'
firstAgg = 'mean'
tbl = all_.groupby(['dset', grouper_kh,'h','fmethod']).agg([firstAgg]).sort_values([grouper_kh, 'dset', 'fmethod']).filter(regex=dset,axis=0)
tbl = tbl.filter(regex=m,axis=0)
renamed = [(map_[c[0]],c[1]) for c in tbl.columns]
tbl.columns = pd.MultiIndex.from_tuples(renamed)
tbl = tbl.drop(columns=("Base", firstAgg))
tbl = tbl.groupby(['dset', 'run', 'fmethod']).agg(['mean'])

if "synthetic" in dset:
    reorder = []
    for index in tbl.index:
        n = int(index[0].split("_")[-1].replace("n",""))
        new = [n, index]
        reorder.append(new)
    reorder = [i[1] for i in sorted(reorder, key=lambda tpls: tpls[0])]
    tbl = tbl.reindex(reorder)

#print(tbl.to_latex(caption="Mean ES Distribution averaged over H's"))
tbl

BU  pMinT     LG
                         mean   mean   mean
                         mean   mean   mean
dset      run  fmethod                     
infantgts kh=1 ets     334.00 319.98 334.68
          kh=h ets     328.17 313.67 318.66

# Improvements

In [133]:
ratios = all_.copy().filter(regex='{0}'.format(variable))
ratiosOverLG = all_.copy().filter(regex='{0}'.format(variable))
baseline = ratios["{0}_BottomUp".format(variable)].copy() # BottomUp as baseline
baselineLG = ratios["{0}_Indep".format(variable)].copy() # LG as baseline
for col in ratios.columns:
    ratios[col] = ratios[col]/baseline
    ratiosOverLG[col] = ratiosOverLG[col]/baselineLG

In [134]:
improvement = (ratios-1)*100
improvementOverLG = (ratiosOverLG-1)*100
cols = ['{0}_Base', '{0}_BottomUp', '{0}_Corr', '{0}_Indep']
cols = [c.format(variable) for c in cols]

In [135]:
pd.options.display.float_format = '{:,.2f}'.format
dset = 'infantgts'
m = 'ets|arima'

firstAgg = 'mean'
tbl = improvement.copy()
tbl = tbl.drop('{0}_Base'.format(variable),axis=1)
tbl = tbl.drop('{0}_BottomUp'.format(variable),axis=1)

tbl = tbl.groupby(['dset', grouper_kh,'h', 'fmethod']).agg([firstAgg]).sort_values([grouper_kh, 'dset','h', 'fmethod']).filter(regex=dset,axis=0)
tbl = tbl.filter(regex=m,axis=0)
agg = tbl.groupby(['dset', grouper_kh, 'fmethod']).agg(['mean']).sort_values(['dset', 'fmethod', 'run'])
renamed = [(map_[c[0]],c[1], c[2]) for c in agg.columns]#[map_[c] for c in tbl.columns] # #
agg.columns = pd.MultiIndex.from_tuples(renamed)
agg = agg.sort_values(['dset', 'run', 'fmethod'])

if "synthetic" in dset:
    reorder = []
    for index in agg.index:
        n = int(index[0].split("_")[-1].replace("n",""))
        new = [n, index]
        reorder.append(new)
    reorder = [i[1] for i in sorted(reorder, key=lambda tpls: tpls[0])]
    agg = agg.reindex(reorder)
    
#print(agg.to_latex(caption='Improvements over BU'))
agg

pMinT    LG
                        mean  mean
                        mean  mean
dset      run  fmethod            
infantgts kh=1 ets     -3.57 -2.97
          kh=h ets     -3.96 -7.42

## Wilcox test

* `pMinT < BU`
* `LG < BU`
* `pMinT < LG`

In [136]:
wilcoxgreater = lambda data: sps.wilcoxon(data, alternative='greater').pvalue
dset = 'infantgts'
m = 'ets|arima'

In [137]:
tbl = improvement.copy()
tbl = tbl.drop('{0}_Base'.format(variable),axis=1)
tbl = tbl.drop('{0}_BottomUp'.format(variable),axis=1)
tbl = tbl.filter(regex=dset, axis=0)
tbl = tbl.filter(regex=m, axis=0)
tbl = tbl.groupby(['dset', grouper_kh,'fmethod']).agg(wilcoxgreater).sort_values(['dset','run', 'fmethod'])
tbl.columns = ['pMinT < BU', 'LG < BU']

In [138]:
tblOverLG = improvementOverLG.copy()
tblOverLG = tblOverLG.drop('{0}_Base'.format(variable),axis=1)
tblOverLG = tblOverLG.drop('{0}_BottomUp'.format(variable),axis=1)
tblOverLG = tblOverLG.drop('{0}_Indep'.format(variable),axis=1)
tblOverLG = tblOverLG.filter(regex=dset, axis=0)
tblOverLG = tblOverLG.filter(regex=m, axis=0)
tblOverLG = tblOverLG.groupby(['dset', grouper_kh,'fmethod']).agg(wilcoxgreater).sort_values(['dset','run', 'fmethod'])
tblOverLG.columns = ['pMinT < LG']

In [139]:
new = pd.merge(left=tbl, right=tblOverLG, how='inner', left_index=True, right_index=True)
#print(new.to_latex(caption='p-values'))

if "synthetic" in dset:
    reorder = []
    for index in new.index:
        n = int(index[0].split("_")[-1].replace("n",""))
        i = [n, index]
        reorder.append(i)
    reorder = [i[1] for i in sorted(reorder, key=lambda tpls: tpls[0])]
    new = new.reindex(reorder)
    
new

pMinT < BU  LG < BU  pMinT < LG
dset      run  fmethod                                 
infantgts kh=1 ets            1.00     1.00        0.85
          kh=h ets            1.00     1.00        0.00